## MACD+Strategy+Backtesting+(c)

In [1]:
import yfinance as yf
import datetime as dt
import matplotlib.pyplot as plt
import pandas as pd 
import pandas_datareader.data as pdr
import numpy as np
import cufflinks as cf
from plotly.offline import download_plotlyjs,init_notebook_mode,plot,iplot
import plotly.graph_objects as go
from plotly.subplots import make_subplots

init_notebook_mode(connected = True)
cf.go_offline()

In [2]:
def calculate_ema(prices, days, smoothing=2):
    ema = [sum(prices[:days]) / days]
    for price in prices[days:]:
        ema.append((price * (smoothing / (1 + days))) + ema[-1] * (1 - (smoothing / (1 + days))))
    return ema

def calculate_macd(prices, fast, slow, smoothing=2):
    fast_ema=pd.DataFrame(calculate_ema(prices,fast),index=np.arange(fast,len(prices)+1),columns=['fast_ema'])
    slow_ema=pd.DataFrame(calculate_ema(prices,slow),index=np.arange(slow,len(prices)+1),columns=['slow_ema'])
    df_macd = pd.concat([fast_ema,slow_ema], axis=1, join="outer")[:-1]
    df_macd['macd']=df_macd['fast_ema']-df_macd['slow_ema']
    df_macd=df_macd[['macd']]
    return df_macd

In [3]:
def macd_backtesting(ticker, fast, slow, previous_days):
    
    start = dt.datetime.today()-dt.timedelta(previous_days)
    end = dt.datetime.today()
    global ohlc_intraday
    ohlc_intraday = {}
    ohlc_intraday[ticker] = yf.download(ticker,start,end)
    ohlc_intraday[ticker] = ohlc_intraday[ticker].reset_index(drop = False)
    df_signal = ohlc_intraday[ticker].copy()
    df_signal = df_signal[['Date','Close']]

    macd=calculate_macd(df_signal["Close"],fast,slow)
    df_signal = pd.concat([df_signal,macd],axis=1,join='outer')[:-1]
    condition =  df_signal['macd'] > 0
    
    df_signal['Hold']=np.where(condition, 'Hold', '')
    df_signal['Hold_t-1']=df_signal['Hold'].shift(1)
    df_signal['Hold_t-2']=df_signal['Hold'].shift(2)

    buy_condition = (df_signal['Hold_t-2'] =='') & (df_signal['Hold_t-1'] =='Hold') 
    df_signal['Buy']=np.where(buy_condition, 'Buy', '')

    sell_condition = (df_signal['Hold_t-2'] =='Hold') & (df_signal['Hold_t-1'] =='') 
    df_signal['Sell']=np.where(sell_condition, 'Sell', '')
    df_signal = df_signal[['Date','Buy','Sell']]
    condition = (df_signal['Buy']=='Buy') | (df_signal['Sell']=='Sell')
    df_signal = df_signal[condition]
    df_signal = df_signal.reset_index(drop = True)
    
    columns_Trading = ['Date_Buy','Price_Buy','Date_Sell','Price_Sell','PL(%)','Holding_Days','Accumulated profit']
    global df_Trading
    df_Trading = pd.DataFrame(columns = columns_Trading)
    
    Accu_profit = 0
    for i in range(0,len(df_signal),2):
        if(i<len(df_signal)-1):
            if (df_signal['Buy'][i]=='Buy') & (df_signal['Sell'][i+1]=='Sell'):

                Date_Buy = df_signal['Date'][i]
                Date_Sell = df_signal['Date'][i+1]
                Price_Buy = ohlc_intraday[ticker][ohlc_intraday[ticker]['Date']==Date_Buy][['Open']].values[0][0]
                Price_Sell = ohlc_intraday[ticker][ohlc_intraday[ticker]['Date']==Date_Sell][['Open']].values[0][0]
                fee = 0.002 * (Price_Buy + Price_Sell)
                Holding_Days = abs(Date_Sell-Date_Buy).days
                PL = (Price_Sell-Price_Buy-fee)*100/Price_Buy
                Accu_profit += PL

                Add_Trading = pd.DataFrame([[Date_Buy,Price_Buy,Date_Sell,Price_Sell,PL,Holding_Days,Accu_profit]],columns = columns_Trading)
                df_Trading = df_Trading.append(Add_Trading)

    df_Trading = df_Trading.reset_index(drop = True)
    df_Trading = df_Trading.round(2)
    
    total_profit = df_Trading['Accumulated profit'][len(df_Trading)-1]
    cagr = ((((total_profit+100)/100)**(1/(len(ohlc_intraday[ticker])/250)))-1)*100
    win_trade = df_Trading[df_Trading["PL(%)"]>0].count()["PL(%)"]
    loss_trade =  df_Trading[df_Trading["PL(%)"]<=0].count()["PL(%)"]
    win_rate = win_trade*100/(win_trade+loss_trade)
    avg_gain = df_Trading[df_Trading["PL(%)"]>0]["PL(%)"].mean()
    avg_loss = df_Trading[df_Trading["PL(%)"]<=0]["PL(%)"].mean()
    expected_return = (win_rate*avg_gain/100)+((1-win_rate)*avg_loss/100)
    list_ = [["Total Profit",round(total_profit,2)],
            ["CAGR",str(round(cagr,2))+'%'],
             ["Win Trade",win_trade],["Loss Trade ",loss_trade],
             ["Win Rate",str(round(win_rate,2))+'%'],["Average Gain",str(round(avg_gain,2))+'%'],
             ["Average loss",str(round(avg_loss,2))+'%'],["Expected Return",str(round(expected_return,2))+'%']]
                      
    global df_summary
    df_summary = pd.DataFrame(list_,columns=['Measurement','Result'])
    
def plot_macd():
    df_plt = ohlc_intraday[ticker].copy()
    macd=calculate_macd(df_plt["Close"],fast,slow)
    df_plt = pd.concat([df_plt,macd],axis=1,join='outer')[:-1]
    df_plt['Date'] = pd.to_datetime(df_plt['Date'], format='%d-%m-%Y')
    df_plt['0']=0
    fig = make_subplots(rows=4, cols=1,shared_xaxes=True, vertical_spacing=0.04,
    specs=[[{"type": "scatter"}],[{"type": "scatter"}],
          [{"type": "table"}],[{"type": "table"}]])

    buy = df_Trading[['Date_Buy','Price_Buy']].copy()
    sell = df_Trading[['Date_Sell','Price_Sell']].copy()
    
    fig.add_trace(go.Candlestick(x=df_plt['Date'],
                open=df_plt['Open'],
                high=df_plt['High'],
                low=df_plt['Low'],
                close=df_plt['Close']),row=1,col=1)

    fig.add_trace(go.Scatter(x = df_plt['Date'], y=df_plt['macd'],mode='lines',name='macd'+str(fast)+','+str(slow)),row= 2,col =1)
    fig.add_trace(go.Scatter(x = df_plt['Date'], y=df_plt['0'],mode='lines'),row= 2,col =1)
    
    fig.add_trace(go.Scatter(x = buy['Date_Buy'], y=buy['Price_Buy'],mode='markers',name='buy'),row= 1,col =1)
    fig.add_trace(go.Scatter(x = sell['Date_Sell'], y=sell['Price_Sell'],mode='markers',name='sell'),row= 1,col =1)
    
    fig.add_trace(go.Table(header=dict(values=df_summary.columns,font=dict(size=10),align="left"),
            cells=dict(values=[df_summary[k].tolist() for k in df_summary.columns],align = "left")),row=3, col=1)

    fig.add_trace(go.Table(header=dict(values=df_Trading.columns,font=dict(size=10),align="left"),
            cells=dict(values=[df_Trading[k].tolist() for k in df_Trading.columns],align = "left")),row=4, col=1)

    fig.update_layout( height=1200, showlegend=True, title_text='MACD')
    fig.update_layout(xaxis_rangeslider_visible=False)
    
    fig.show()

In [4]:
ticker ='AAPL'
fast = 15
slow = 30
macd_backtesting(ticker, fast = fast, slow = slow, previous_days =4000)

[*********************100%***********************]  1 of 1 completed


In [5]:
plot_macd()

In [6]:
# 1. Download Date from yfinance
ticker = 'AAPL'
start = dt.datetime.today()-dt.timedelta(4000)
end = dt.datetime.today()
ohlc_intraday = {}
ohlc_intraday[ticker] = yf.download(ticker,start,end)
ohlc_intraday[ticker] = ohlc_intraday[ticker].reset_index(drop = False)
df_signal = ohlc_intraday[ticker].copy()
df_signal

[*********************100%***********************]  1 of 1 completed


,Date,Open,High,Low,Close,Adj Close,Volume
0,2011-01-04,11.872857,11.875000,11.719643,11.831786,10.144160,309080800
1,2011-01-05,11.769643,11.940714,11.767857,11.928571,10.227139,255519600
2,2011-01-06,11.954286,11.973214,11.889286,11.918929,10.218874,300428800
3,2011-01-07,11.928214,12.012500,11.853571,12.004286,10.292056,311931200
4,2011-01-10,12.101071,12.258214,12.041786,12.230357,10.485882,448560000
...,...,...,...,...,...,...,...
2753,2021-12-10,175.210007,179.630005,174.690002,179.449997,179.449997,115228100
2754,2021-12-13,181.119995,182.130005,175.529999,175.740005,175.740005,153237000
2755,2021-12-14,175.250000,177.740005,172.210007,174.330002,174.330002,139380400
2756,2021-12-15,175.110001,179.500000,172.309998,179.300003,179.300003,131063300


In [7]:
# 2.remove some columns
df_signal = df_signal[['Date','Close']]
df_signal

,Date,Close
0,2011-01-04,11.831786
1,2011-01-05,11.928571
2,2011-01-06,11.918929
3,2011-01-07,12.004286
4,2011-01-10,12.230357
...,...,...
2753,2021-12-10,179.449997
2754,2021-12-13,175.740005
2755,2021-12-14,174.330002
2756,2021-12-15,179.300003


In [8]:
# 3.Add macd
fast = 20
slow = 40
macd=calculate_macd(df_signal["Close"],fast,slow)
df_signal = pd.concat([df_signal,macd],axis=1,join='outer')[:-1]
df_signal

,Date,Close,macd
0,2011-01-04,11.831786,NaN
1,2011-01-05,11.928571,NaN
2,2011-01-06,11.918929,NaN
3,2011-01-07,12.004286,NaN
4,2011-01-10,12.230357,NaN
...,...,...,...
2752,2021-12-09,174.559998,4.840351
2753,2021-12-10,179.449997,5.226162
2754,2021-12-13,175.740005,5.761100
2755,2021-12-14,174.330002,6.022360


In [9]:
# 4. Find days 'hold'
condition =  df_signal['macd'] > 0
df_signal['Hold']=np.where(condition, 'Hold', '')
df_signal

,Date,Close,macd,Hold
0,2011-01-04,11.831786,NaN,
1,2011-01-05,11.928571,NaN,
2,2011-01-06,11.918929,NaN,
3,2011-01-07,12.004286,NaN,
4,2011-01-10,12.230357,NaN,
...,...,...,...,...
2752,2021-12-09,174.559998,4.840351,Hold
2753,2021-12-10,179.449997,5.226162,Hold
2754,2021-12-13,175.740005,5.761100,Hold
2755,2021-12-14,174.330002,6.022360,Hold


In [10]:
# 5. Find 'buy','sell' day

df_signal['Hold_t-1']=df_signal['Hold'].shift(1)
df_signal['Hold_t-2']=df_signal['Hold'].shift(2)

buy_condition = (df_signal['Hold_t-2'] =='') & (df_signal['Hold_t-1'] =='Hold') 
df_signal['Buy']=np.where(buy_condition, 'Buy', '')

sell_condition = (df_signal['Hold_t-2'] =='Hold') & (df_signal['Hold_t-1'] =='') 
df_signal['Sell']=np.where(sell_condition, 'Sell', '')

df_signal

,Date,Close,macd,Hold,Hold_t-1,Hold_t-2,Buy,Sell
0,2011-01-04,11.831786,NaN,,NaN,NaN,,
1,2011-01-05,11.928571,NaN,,,NaN,,
2,2011-01-06,11.918929,NaN,,,,,
3,2011-01-07,12.004286,NaN,,,,,
4,2011-01-10,12.230357,NaN,,,,,
...,...,...,...,...,...,...,...,...
2752,2021-12-09,174.559998,4.840351,Hold,Hold,Hold,,
2753,2021-12-10,179.449997,5.226162,Hold,Hold,Hold,,
2754,2021-12-13,175.740005,5.761100,Hold,Hold,Hold,,
2755,2021-12-14,174.330002,6.022360,Hold,Hold,Hold,,


In [11]:
# 6. remove some columns
df_signal = df_signal[['Date','Buy','Sell']]
df_signal

,Date,Buy,Sell
0,2011-01-04,,
1,2011-01-05,,
2,2011-01-06,,
3,2011-01-07,,
4,2011-01-10,,
...,...,...,...
2752,2021-12-09,,
2753,2021-12-10,,
2754,2021-12-13,,
2755,2021-12-14,,


In [12]:
# 7. filter days sending order
condition = (df_signal['Buy']=='Buy') | (df_signal['Sell']=='Sell')

df_signal = df_signal[condition]
df_signal = df_signal.reset_index(drop = True)
df_signal

,Date,Buy,Sell
0,2011-03-04,Buy,
1,2011-03-23,,Sell
2,2011-03-30,Buy,
3,2011-04-07,,Sell
4,2011-05-03,Buy,
5,2011-05-19,,Sell
6,2011-07-11,Buy,
7,2011-11-21,,Sell
8,2011-12-27,Buy,
9,2012-05-18,,Sell


In [13]:
# 8. Make trading dataframe
columns_Trading = ['Date_Buy','Price_Buy','Date_Sell','Price_Sell','PL(%)','Holding_Days','Accumulated profit']
df_Trading = pd.DataFrame(columns = columns_Trading)
df_Trading

,Date_Buy,Price_Buy,Date_Sell,Price_Sell,PL(%),Holding_Days,Accumulated profit


In [14]:
# 9. fill data in trading dataframe

Accu_profit = 0
for i in range(0,len(df_signal),2):
    if(i<len(df_signal)-1):
        if (df_signal['Buy'][i]=='Buy') & (df_signal['Sell'][i+1]=='Sell'):

            Date_Buy = df_signal['Date'][i]
            Date_Sell = df_signal['Date'][i+1]
            Price_Buy = ohlc_intraday[ticker][ohlc_intraday[ticker]['Date']==Date_Buy][['Open']].values[0][0]
            Price_Sell = ohlc_intraday[ticker][ohlc_intraday[ticker]['Date']==Date_Sell][['Open']].values[0][0]
            fee = 0.002 * (Price_Buy + Price_Sell)
            Holding_Days = abs(Date_Sell-Date_Buy).days
            PL = (Price_Sell-Price_Buy-fee)*100/Price_Buy
            Accu_profit += PL

            Add_Trading = pd.DataFrame([[Date_Buy,Price_Buy,Date_Sell,Price_Sell,PL,Holding_Days,Accu_profit]],columns = columns_Trading)
            df_Trading = df_Trading.append(Add_Trading)

df_Trading = df_Trading.reset_index(drop = True)
df_Trading = df_Trading.round(2)
df_Trading

,Date_Buy,Price_Buy,Date_Sell,Price_Sell,PL(%),Holding_Days,Accumulated profit
0,2011-03-04,12.86,2011-03-23,12.12,-6.16,19,-6.16
1,2011-03-30,12.52,2011-04-07,12.07,-3.97,8,-10.13
2,2011-05-03,12.43,2011-05-19,12.22,-2.09,16,-12.23
3,2011-07-11,12.73,2011-11-21,13.23,3.54,133,-8.69
4,2011-12-27,14.40,2012-05-18,19.07,32.00,143,23.31
5,2012-06-20,21.01,2012-10-17,23.17,9.89,119,33.20
6,2013-05-13,16.13,2013-06-18,15.41,-4.81,36,28.39
7,2013-07-30,16.07,2014-01-31,17.68,9.63,185,38.02
8,2014-03-27,19.29,2014-04-14,18.64,-3.75,18,34.27
9,2014-04-28,20.46,2015-01-20,26.96,31.32,267,65.60


In [15]:
#10. Trading Summary
total_profit = df_Trading['Accumulated profit'][len(df_Trading)-1]
cagr = ((((total_profit+100)/100)**(1/(len(ohlc_intraday[ticker])/250)))-1)*100
win_trade = df_Trading[df_Trading["PL(%)"]>0].count()["PL(%)"]
loss_trade =  df_Trading[df_Trading["PL(%)"]<=0].count()["PL(%)"]
win_rate = win_trade*100/(win_trade+loss_trade)
avg_gain = df_Trading[df_Trading["PL(%)"]>0]["PL(%)"].mean()
avg_loss = df_Trading[df_Trading["PL(%)"]<=0]["PL(%)"].mean()
expected_return = (win_rate*avg_gain/100)+((1-win_rate)*avg_loss/100)
list_ = [["Total Profit",round(total_profit,2)],
            ["CAGR",str(round(cagr,2))+'%'],
             ["Win Trade",win_trade],["Loss Trade ",loss_trade],
             ["Win Rate",str(round(win_rate,2))+'%'],["Average Gain",str(round(avg_gain,2))+'%'],
             ["Average loss",str(round(avg_loss,2))+'%'],["Expected Return",str(round(expected_return,2))+'%']]
df_summary = pd.DataFrame(list_,columns=['Measurement','Result'])

In [16]:
#11.Plot graph

df_plt = ohlc_intraday[ticker].copy()
macd=calculate_macd(df_plt["Close"],fast,slow)
df_plt = pd.concat([df_plt,macd],axis=1,join='outer')[:-1]
df_plt['Date'] = pd.to_datetime(df_plt['Date'], format='%d-%m-%Y')
df_plt['0']=0

fig = make_subplots(rows=4, cols=1,shared_xaxes=True, vertical_spacing=0.04,
specs=[[{"type": "scatter"}],[{"type": "scatter"}],
      [{"type": "table"}],[{"type": "table"}]])

buy = df_Trading[['Date_Buy','Price_Buy']].copy()
sell = df_Trading[['Date_Sell','Price_Sell']].copy()

fig.add_trace(go.Candlestick(x=df_plt['Date'],
            open=df_plt['Open'],
            high=df_plt['High'],
            low=df_plt['Low'],
            close=df_plt['Close']),row=1,col=1)

fig.add_trace(go.Scatter(x = df_plt['Date'], y=df_plt['macd'],mode='lines',name='macd'+str(fast)+','+str(slow)),row=2,col=1)
fig.add_trace(go.Scatter(x = df_plt['Date'], y=df_plt['0'],mode='lines'),row= 2,col =1)

fig.add_trace(go.Scatter(x = buy['Date_Buy'], y=buy['Price_Buy'],mode='markers',name='buy'),row= 1,col =1)
fig.add_trace(go.Scatter(x = sell['Date_Sell'], y=sell['Price_Sell'],mode='markers',name='sell'),row= 1,col =1)

fig.add_trace(go.Table(header=dict(values=df_summary.columns,font=dict(size=10),align="left"),
        cells=dict(values=[df_summary[k].tolist() for k in df_summary.columns],align = "left")),row=3, col=1)

fig.add_trace(go.Table(header=dict(values=df_Trading.columns,font=dict(size=10),align="left"),
        cells=dict(values=[df_Trading[k].tolist() for k in df_Trading.columns],align = "left")),row=4, col=1)

fig.update_layout( height=1200, showlegend=True, title_text='MACD')
fig.update_layout(xaxis_rangeslider_visible=False)

fig.show()